In [1]:
import pandas as pd
import numpy as np
import schedule
import os
import math
import datetime
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt
import json
from polygeohasher import polygeohasher
from shapely.geometry import Point, Polygon
import plotly.express as px
import datetime as dt

In [2]:
allSanFran = pd.read_csv('./cabs.csv')

In [3]:
allSanFran

,driver,lat,lon,occupied,timestamp
0,egteir,37.75135,-122.39526,0,1212567954
1,egteir,37.75143,-122.39529,0,1212567848
2,egteir,37.75154,-122.39531,0,1212567624
3,egteir,37.75154,-122.39531,0,1212567577
4,egteir,37.75159,-122.39530,0,1212567509
...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,1211030268
11219951,ashgati,37.75127,-122.39441,0,1211030171
11219952,ashgati,37.75154,-122.39453,0,1211030106
11219953,ashgati,37.75157,-122.39452,0,1211030045


In [7]:
#come prima cosa ho da modificare il formato di timestamp e convertirlo in datetime
allSanFran['timestamp'] = pd.to_datetime(allSanFran['timestamp'], unit='s')

allSanFran

#primo problema, ho il dataset al contrario!
#per prima cosa troviamo l'area di appartenza con geohash

,driver,lat,lon,occupied,timestamp
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29
...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05


In [10]:
import pygeohash as gh

allSanFran['geohash_area']=allSanFran.apply(lambda x: gh.encode(x.lat, x.lon, precision=5), axis=1)


KeyboardInterrupt: 

In [28]:
#parte di visualizzazione
import geopandas as gpd

gdf = gpd.read_file('./SanFranciscoBoun.geojson')
primary_df = polygeohasher.create_geohash_list(gdf,5,inner=False)
geo_df = polygeohasher.geohashes_to_geometry(primary_df,"geohash_list")
geo_df.to_file("SanFranciscoGrid.geojson",driver = "GeoJSON")
SanFranGrid = json.load(open('SanFranciscoGrid.geojson','r'))


In [29]:
import folium
m = folium.Map([37.80, -112.26], zoom_start=12, tiles='cartodbpositron')
folium.GeoJson(SanFranGrid, name="geojson").add_to(m)
m

In [7]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yy
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yy
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yy
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yy
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yy
...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yy
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yy
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yy
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yy


In [8]:
#Molte meno celle rispetto a Pechino, chiedere se per caso sia meglio utilizzare geohashing 6 caratteri invece di 5
#potrebbe essere fatto in poco tempo, nel frattempo inizio con la velocità e la distanza.
#altro problema è che il dataset è al contrario molto probabilmente.





Da quello che ho capito aprendo il dataset, sono registrati tot. guidatori per i giorni che vanno dal 17-05 al 06-04, con un ordine cronologico che va dal più recente al meno recente (Il contrario di quello di Pechino).

Possbili idea: scorrere il dataset al contraio -> la più intelligente, non dovrei avere grossi problemi.


In [10]:
latlonSanFran=allSanFran[['driver','lon','lat']].to_numpy() #copio la lista di coordinate e di driver in questo array

In [11]:
#iniziamo a calcolare la distanza tra ogni punto

#Funzione che mi implementa la forumla di Haversine per calcolare la distanza che c'è tra due punti successivi
from math import radians, cos, sin, asin, sqrt
def measure(lat1, lon1, lat2, lon2) :
    
    R = 6372.8 

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

In [12]:
#Calcolo la distanza che c'è tra ogni punto al fine di potermi calcolare la velocità

i = len(latlonSanFran)-1
lista = [] 
id= latlonSanFran[i][0]
while(i>=0):
    if(id!=latlonSanFran[i-1][0]):
        
        lista.append('end')
        id=latlonSanFran[i-1][0]
        
    else:
        lista.append(measure(latlonSanFran[i-1][2],latlonSanFran[i-1][1],latlonSanFran[i][2],latlonSanFran[i][1])*1000)
    i=i-1

In [13]:
lista1 = lista.reverse() #mi permette di rovesciare la lista delle distanze

In [14]:
allSanFran['distances_m'] = lista

In [15]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yy,end
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yy,9.281003
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yy,12.360678
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yy,0.0
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yy,5.630422
...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yy,98.898318
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yy,36.95322
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yy,31.831416
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yy,3.450736


Trovata la distanza tra ogni punto, adesso mi serve trovare gli intervalli di tempo tra ogni coppia di record

In [16]:
#funzione che permette di trovare la differenza tra due datetime
def timeSet(datetime1,datetime2):
    fmt = '%Y-%d-%m %H:%M:%S'
    tstamp1 = datetime.strptime(datetime1, fmt)
    tstamp2 = datetime.strptime(datetime2, fmt)

    if tstamp1 > tstamp2:
        td = tstamp1 - tstamp2
    else:
        td = tstamp2 - tstamp1
    return td.total_seconds()

In [17]:
record = allSanFran[['driver','timestamp']].to_numpy()

In [18]:
#funzione che mi permette di trovare gli intervalli di registrazione tra ogni coppia di record
def ctime(latlonSanFran):
    result = []
    intervaltime=len(latlonSanFran)-1
    i = intervaltime
    id=latlonSanFran[i][0]
    while (i>=0):
        if(id!=latlonSanFran[i-1][0]):
            result.append('end')
            id=latlonSanFran[i-1][0]
        elif(timeSet(latlonSanFran[i-1][1].strftime("%Y-%d-%m %H:%M:%S"),latlonSanFran[i][1].strftime("%Y-%d-%m %H:%M:%S")) > 3600.0):
            result.append("sosta")
        else:
            result.append(timeSet(latlonSanFran[i-1][1].strftime("%Y-%d-%m %H:%M:%S"),latlonSanFran[i][1].strftime("%Y-%d-%m %H:%M:%S")))
        i=i-1;
    return result

In [19]:
resultRecordTime = ctime(record)

In [20]:
resultRecordTime1 = resultRecordTime.reverse()

In [21]:
allSanFran['IntervalRecordTime_s'] = resultRecordTime

In [22]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m,IntervalRecordTime_s
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yy,end,end
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yy,9.281003,106.0
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yy,12.360678,224.0
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yy,0.0,47.0
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yy,5.630422,68.0
...,...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yy,98.898318,61.0
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yy,36.95322,97.0
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yy,31.831416,65.0
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yy,3.450736,61.0


Trovato l'intervallo di tempo, adesso posso calcolare la velocità tra ogni punto, successivamente l'accelerazione

In [23]:
#funzione che mi permette di calcolare la velocità tra ogni record 
def measure_speed(lista,resultRecordTime):
    intervalSpeed=len(resultRecordTime)-1 #lista and timex have same lenght
    speedList= []
    i=intervalSpeed
    while (i>=0):
        if(resultRecordTime[i] == "sosta" or resultRecordTime[i]=="end" or resultRecordTime[i]=="end" or resultRecordTime[i]==0):
            speedList.append(0.0)
        elif((lista[i]/(resultRecordTime[i]))*3.6 > 150.0):
            speedList.append("assurdo")
        else:
            speedList.append((lista[i]/(resultRecordTime[i]))*3.6) #in k/h
        i=i-1    
    return speedList

In [24]:
resultSpeed = measure_speed(lista,resultRecordTime)

In [25]:
resultSpeed1 = resultSpeed.reverse()

In [26]:
allSanFran['speed_km'] = resultSpeed

In [27]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m,IntervalRecordTime_s,speed_km
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yy,end,end,0.0
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yy,9.281003,106.0,0.315204
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yy,12.360678,224.0,0.198654
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yy,0.0,47.0,0.0
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yy,5.630422,68.0,0.298081
...,...,...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yy,98.898318,61.0,5.836622
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yy,36.95322,97.0,1.37146
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yy,31.831416,65.0,1.762971
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yy,3.450736,61.0,0.20365


Devo trovare gli starting point e gli ending point di ogni veicolo

Quasi tutti i veicoli sono registrati tutto il giorno per tutti i giorni, magari prendo ending point che hanno una sosta maggiore di 1h

In [28]:
def start(allSanFran):
    i = len(allSanFran)-1
    lon = [allSanFran['lon'][i]]
    lat = [allSanFran['lat'][i]]
    driver = [allSanFran['driver'][i]]
    
    while(i>=0):
        if((allSanFran['IntervalRecordTime_s'][i] == 'sosta' or allSanFran['distances_m'][i]== 'end')and i !=0):
            lon.append(allSanFran['lon'][i-1])
            lat.append(allSanFran['lat'][i-1])
            driver.append(allSanFran['driver'][i-1])
        i = i-1
    list_of_tuples = list(zip(driver,lon,lat))
    list_of_tuples
    startDf = pd.DataFrame(list_of_tuples,columns = ['driver','longitude','latitude'])
    return startDf

def end(allSanFran):
    i = len(allSanFran)-1
    lon = []
    lat = []
    driver = []
    
    while(i>=0):
        if(allSanFran['IntervalRecordTime_s'][i] == 'sosta' or allSanFran['distances_m'][i]== 'end'):
            lon.append(allSanFran['lon'][i])
            lat.append(allSanFran['lat'][i])
            driver.append(allSanFran['driver'][i])
        i = i-1
    list_of_tuples = list(zip(driver,lon,lat))
    list_of_tuples
    endDf = pd.DataFrame(list_of_tuples,columns = ['driver','longitude','latitude'])
    return endDf

In [29]:
allStart = start(allSanFran)
allEnd = end(allSanFran)

In [30]:
allStart

,driver,longitude,latitude
0,ashgati,-122.39448,37.75155
1,ashgati,-122.40603,37.77532
2,ashgati,-122.39364,37.75149
3,ashgati,-122.39404,37.75056
4,ashgati,-122.39422,37.75218
...,...,...,...
18302,egteir,-122.38587,37.61657
18303,egteir,-122.38827,37.61609
18304,egteir,-122.39533,37.75035
18305,egteir,-122.40296,37.74823


In [31]:
allEnd

,driver,longitude,latitude
0,ashgati,-122.39527,37.75158
1,ashgati,-122.39528,37.75144
2,ashgati,-122.39388,37.75062
3,ashgati,-122.39418,37.75193
4,ashgati,-122.39432,37.75192
...,...,...,...
18302,egteir,-122.39278,37.61445
18303,egteir,-122.41899,37.76991
18304,egteir,-122.39515,37.75116
18305,egteir,-122.39264,37.75134


In [32]:
#geohashing start e end dtaframes

allStart['geohash_area']=allStart.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=5), axis=1)
allEnd['geohash_area']=allEnd.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=5), axis=1)


In [33]:
#creo gli slot temporali per verificare quali sono le ore con maggior numero di veicoli
allSanFran['timestamp']=pd.to_datetime(allSanFran['timestamp'],format='%Y-%d-%m %H:%M:%S')
allSanFran=allSanFran.assign(session=pd.cut(allSanFran.timestamp.dt.hour,
                            [0,6,12,18,23],
                            labels=['Night','Morning','Afternoon','Evening'],
                            include_lowest=True))

In [34]:
allSanFran.to_csv('./allSanFran.csv')

Posso iniziare adesso a considerare le aree

In [30]:
from shapely import wkt
geo_df.to_csv("Tassellation.csv") #solito problema per ordinare il geodataframe
tasselation = pd.read_csv("./Tassellation.csv")
del tasselation["Unnamed: 0"]
tasselation['geometry'] = tasselation['geometry'].apply(wkt.loads)
geo_tassellation = gpd.GeoDataFrame(
    tasselation,geometry='geometry')
#risolto problema

In [36]:
allArea = pd.DataFrame(geo_tassellation['geohash_list']) #Dataframe della griglia

In [6]:
allSanFran = allSanFran.drop(allSanFran[allSanFran.speed_km == 'assurdo'].index) #alcune coordinate sono errate

In [38]:
#Funzione che conta i numero di punti per area
def Count1(allArea,allS):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j=0
        areaStart = allS[allS.geohash_area == allArea['geohash_list'][i]]
        result.append(len(areaStart['geohash_area']))
        i=i+1
    return result

In [39]:
resultCountStart = Count1(allArea,allStart) #conto numero veicoli in partenza da un'area
resultCountEnd = Count1(allArea,allEnd) #conto numero di veicoli in arrivo in un area
resultCountTot = Count1(allArea,allSanFran) #conto numero di veicoli che transitano in un area per spazio temporale pari a 3 settimane

In [40]:
allArea['StartingVehicles']= resultCountStart
allArea['EndingVehicles'] = resultCountEnd
allArea['TotNumberRecordVehicles'] = resultCountTot

In [41]:
#funzione che trova la velocità massima di una determinata area i
def SpeedMax(allSanFran,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        speed = area['speed_km']
        result.append(speed.max())
        i=i+1
    return result

In [42]:
resultSpeedMax = SpeedMax(allSanFran,allArea)

In [43]:
allArea['maxSpeed_km/h'] = resultSpeedMax #aggiungo l'attributo velocità al dataframe

In [44]:
#funzione che mi ritorna la velocità media di una determinata area
def AverageSpeed(allSanFran,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        speed = area['speed_km']
        result.append(speed.mean())
        i=i+1
    return result

In [45]:
resultAverageSpeed= AverageSpeed(allSanFran,allArea)

In [46]:
allArea['AverageSpeed'] = resultAverageSpeed #aggiungo l'attributo velocità media al dataframe

In [47]:
allStart['end'] = allEnd['geohash_area'] #aggiorno il dataframe di inzio con l'attributo end
allEnd['start'] = allStart['geohash_area'] #aggiorno il dataframe di fine con l'attributo start

In [48]:
#cerco di vedere quanti sono i veicoli entranti ed i veicoli uscenti in una determinata area 
#uscenti -> iniziano in un area specificat e terminano la corsa in un'altra area
#entranti -> iniziano la corsa in una certa area e terminano la corsa in una specifica area


def checkOutEdge(allArea,allStart):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j=0
        count=0
        areaStart = allStart[allStart.geohash_area == allArea['geohash_list'][i]]
        if(len(areaStart['geohash_area'])==0):
            result.append(count)
            
        else:
            while(j<len(areaStart['geohash_area'])):
                new =areaStart.reset_index()
                del new['index']
                if(new['geohash_area'][j]!= new['end'][j]):
                    count = count+1
                j=j+1
            result.append(count)
            
        i=i+1
    return result



def checkInEdge(allArea,allEnd):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j = 0
        count = 0
        areaEnd = allEnd[allEnd.geohash_area == allArea['geohash_list'][i]]
        if(len(areaEnd)== 0):
            result.append(count)
        else:
            while(j<len(areaEnd['geohash_area'])):
                new = areaEnd.reset_index()
                del new['index']
                if(new['geohash_area'][j]!= new['start'][j]):
                    count = count+1
                j=j+1
            result.append(count)
        i=i+1
    return result

In [49]:
resultOutEdges = checkOutEdge(allArea,allStart)
resultInEdges = checkInEdge(allArea,allEnd)

In [50]:
#aggiunti attributi archi entranti ed archi uscenti
allArea['OutEdges'] = resultOutEdges
allArea['InEdges'] = resultInEdges

In [52]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m,IntervalRecordTime_s,speed_km,session
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yy,end,end,0.0,Morning
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yy,9.281003,106.0,0.315204,Morning
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yy,12.360678,224.0,0.198654,Morning
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yy,0.0,47.0,0.0,Morning
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yy,5.630422,68.0,0.298081,Morning
...,...,...,...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yy,98.898318,61.0,5.836622,Afternoon
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yy,36.95322,97.0,1.37146,Afternoon
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yy,31.831416,65.0,1.762971,Afternoon
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yy,3.450736,61.0,0.20365,Afternoon


In [53]:
#Funzione che mi calcola il numero di punti per slot temporale
def coutHour(allArea,allSanFran,slot):
    i = 0
    result = []
    allAreaLen = len(allArea['geohash_list'])
    allSanFranLen = len(allSanFran['session'])
    while(i<allAreaLen):
        j=0
        area = allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        area1= area[area.session == slot]
        result.append(len(area1['geohash_area']))
        i=i+1
    return result

In [54]:
resultMorning = coutHour(allArea,allSanFran,'Morning')
resultAfternoon = coutHour(allArea,allSanFran,'Afternoon')
resultEvening = coutHour(allArea,allSanFran,'Evening')
resultNight = coutHour(allArea,allSanFran,'Night')

In [55]:
allArea['VehiclesNight(24-5 am)'] = resultNight
allArea['VehiclesMorning(6-12 am)'] = resultMorning
allArea['VehiclesAfternoon(13-18 pm)'] = resultAfternoon
allArea['VehiclesEvening(19-23 pm)'] =resultEvening

In [57]:
allArea.to_csv('sanFrancisco.csv')

In [59]:
allStart.to_csv('./allStart.csv')

In [62]:
allEnd.to_csv('./allEnd.csv')

In [ ]:
allSanFran.to_csv('./allSanFran.csv')

In [64]:
#copia effettuata

In [3]:
allArea = pd.read_csv('sanFrancisco.csv')
allStart = pd.read_csv('allStart.csv')
allEnd = pd.read_csv('allEnd.csv')
allSanFran = pd.read_csv('allSanFran.csv') 

In [4]:
del allArea['Unnamed: 0']
del allStart['Unnamed: 0']
del allEnd['Unnamed: 0']
del allSanFran['Unnamed: 0']

In [5]:
#funzione che mi calcola la densità di punti per area( Area = 4.89*4.89)
def Density(allArea):
    i = 0 
    result = []
    areaLen = len(allArea['TotNumberRecordVehicles'])
    while(i<areaLen):
        count = (allArea['TotNumberRecordVehicles'][i])/(4.89*4.89)
        result.append(count)
        i=i+1
    return result

In [6]:
resultDensity = Density(allArea) #densità punti per area

In [7]:
allArea['Density_n/km^2'] = resultDensity #aggiorno il dataframe con l'attributo denistà

In [8]:
allSanFran['speed_m/s'] = allSanFran['speed_km']/3.6

In [9]:
#cambiare il tipo di oggetto della classe record time
def changetype(allBeijing):
    
    i = 0
    result = []
    allBeijingLen  = len(allBeijing['IntervalRecordTime_s'])
    while(i<allBeijingLen):
        if(allBeijing['IntervalRecordTime_s'][i]=='sosta' or allBeijing['IntervalRecordTime_s'][i]=='end'):
            result.append(0.0)
        else:
            result.append(float(allBeijing['IntervalRecordTime_s'][i]))
        i=i+1
    return result

In [10]:
resultRecordTime = changetype(allSanFran) #cambio il tipo della colonna

In [11]:
allSanFran['recordTime_float'] = resultRecordTime #aggiorno dataframe SanFrancisco

In [12]:
allSanFran['acceleration_m/s^2'] = allSanFran['speed_m/s'] / allSanFran['recordTime_float']

In [14]:
def AccMax(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        acceleration = area['acceleration_m/s^2']
        result.append(acceleration.max())
        i=i+1
    return result

In [16]:
resultAcceleration = AccMax(allSanFran,allArea)

In [17]:
allArea['AccelarationMax'] = resultAcceleration

In [18]:
def AccAverage(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        acceleration = area['acceleration_m/s^2']
        result.append(acceleration.mean())
        i=i+1
    return result

In [19]:
resultAverageAcc = AccAverage(allSanFran,allArea)

In [20]:
allArea['AverageAcceleration_m/s^2'] = resultAverageAcc

In [23]:
allArea.to_csv('./sanFrancisco.csv')

Inizio con i punti di interesse

In [33]:
import geopandas as gpd

museum = gpd.read_file('./interessi/museumS.geojson')
attraction = gpd.read_file('./interessi/attractionS.geojson')
university = gpd.read_file('./interessi/universityS.geojson')
bank = gpd.read_file('./interessi/bankS.geojson')
hospital = gpd.read_file('./interessi/hospitalS.geojson')
dentist = gpd.read_file('./interessi/dentistS.geojson')
cinema = gpd.read_file('./interessi/cinemaS.geojson')
stadium = gpd.read_file('./interessi/stadiumS.geojson')
fuel = gpd.read_file('./interessi/fuelS.geojson')
zoo = gpd.read_file('./interessi/zooS.geojson')
pharmacy = gpd.read_file('./interessi/pharmacyS.geojson')
busStation = gpd.read_file('./interessi/bus_stationS.geojson')
ruins = gpd.read_file('./interessi/ruinsS.geojson')
swimmingPool = gpd.read_file('./interessi/swimmingPoolS.geojson')
taxi_stop = gpd.read_file('./interessi/taxiS.geojson')
company = gpd.read_file('./interessi/company.geojson')
library = gpd.read_file('./interessi/libraryS.geojson')
theatre = gpd.read_file('./interessi/theatreS.geojson')

In [34]:
def checkInterest(allArea,interest):
    i = 0
    count = 0
    result = []
    interestLen = len(interest['geometry'])
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j = 0
        count = 0
        while(j<interestLen):
            if(interest['geometry'][j].centroid.within(allArea['geometry'][i])):
                count = count+1
            
            j=j+1
        result.append(count)    
        i=i+1
    return result

In [35]:
resultMuseum = checkInterest(geo_tassellation,museum)
resultAttraction = checkInterest(geo_tassellation,attraction)
resultUniversity= checkInterest(geo_tassellation,university)
resultBank = checkInterest(geo_tassellation,bank)
resultHospital = checkInterest(geo_tassellation,hospital)
resultDentist = checkInterest(geo_tassellation,dentist)
resultCinema = checkInterest(geo_tassellation,cinema)
resultStadium = checkInterest(geo_tassellation,stadium)
resultPharmacy = checkInterest(geo_tassellation,pharmacy)
resultFuel = checkInterest(geo_tassellation,fuel)
resultZoo = checkInterest(geo_tassellation,zoo)
resultBus = checkInterest(geo_tassellation,busStation)
resultRuins = checkInterest(geo_tassellation,ruins)
resultSP = checkInterest(geo_tassellation,swimmingPool)
resultTaxi = checkInterest(geo_tassellation,taxi_stop)
resultCompany = checkInterest(geo_tassellation,company)
resultLibrary = checkInterest(geo_tassellation,library)
resultTheatre = checkInterest(geo_tassellation,theatre)

In [36]:
resultCulture= []
i = 0

while(i<len(resultAttraction)):
    count = 0
    count = resultMuseum[i]+ resultAttraction[i]+resultRuins[i]
    resultCulture.append(count)
    i=i+1

In [37]:
resultFinance= []
i =0

while(i<len(resultBank)):
    count = 0
    count = resultBank[i] + resultCompany[i]
    resultFinance.append(count)
    i=i+1

In [38]:
resultEnterteiment= []
i =0

while(i<len(resultCinema)):
    count = 0
    count = resultCinema[i] + resultTheatre[i] + resultZoo[i] + resultStadium[i] +resultSP[i]
    resultEnterteiment.append(count)
    i=i+1

In [39]:
resultTrasportation = []
i = 0
while(i<len(resultBus)):
    count = 0
    count = resultBus[i] +resultFuel[i] + resultTaxi[i]
    resultTrasportation.append(count)
    i = i+1

In [40]:
resultEducation = []
i = 0
while(i<len(resultLibrary)):
    count = 0
    count = resultUniversity[i] + resultLibrary[i]
    resultEducation.append(count)
    i=i+1

In [42]:
resultHealth= []
i =0

while(i<len(resultHospital)):
    count = 0
    count = resultHospital[i] + resultDentist[i] + resultPharmacy[i]
    resultHealth.append(count)
    i=i+1

In [43]:
#mi sono scordato il numero totale di punti di interesse

i = 0
totalInterest = []
while(i<len(resultBus)):
    count = 0
    count = resultCulture[i]+ resultFinance[i]+resultEnterteiment[i]+resultHealth[i]+resultTrasportation[i]+resultEducation[i]
    totalInterest.append(count)
    i=i+1


In [44]:
allArea['NumberTotalPointInterest'] = totalInterest
allArea['NumberCulturalPoint("Museum-Attraction-Ruins")'] = resultCulture
allArea['NumberFinancePoint("Bank-Company")'] = resultBank
allArea['NumberHealthPoint("Hospital-Dentist-Pharmacy")'] = resultHealth
allArea['NumberTrasportationPoint("Fuel-BusStation-TaxiStop")'] = resultTrasportation
allArea['NumberEducationPoint("University-Library")'] = resultEducation
allArea['NumberEnterteimentPoint("Cinema-Theatre-Zoo-Stadium-SwimmingPool")'] = resultEnterteiment

In [46]:
allArea.to_csv('sanFrancisco.csv')

Verificare il numero di punti durante la settimana

In [49]:
allSanFran['timestamp']=pd.to_datetime(allSanFran['timestamp'],format='%Y-%m-%d %H:%M:%S')

In [51]:
#utilizzo la funzione weekday che mi permette di assegnare un numero da 0 a 6 per indicare il giorno della settimana
i = 0
result = []
l = len(allSanFran['timestamp'])
while(i<l):
    day = allSanFran['timestamp'][i].weekday()
    result.append(day)
    i = i+1

In [52]:
allSanFran['weekday'] = result

In [54]:
allSanFran.to_csv('./allSanFran.csv')

In [58]:
#funzione che mi calcola il numero di punti per area nel weekend
def checkWeek(allArea,allSanFran):
    i = 0
    result = []
    lenArea = len(allArea['geohash_list'])
    while(i<lenArea):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        area1 = area[area.weekday == 5]
        area2 = area[area.weekday == 6]
        result.append(len(area1)+len(area2))    
        i=i+1
    return result

In [59]:
resultWeek = checkWeek(allArea,allSanFran)

In [60]:
allArea['RecordWeekend'] = resultWeek

In [68]:
def checkWeekday(allArea,allSanFran):
    i = 0
    result = []
    lenArea = len(allArea['geohash_list'])
    while(i<lenArea):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        area1 = area[area.weekday == 0]
        area2 = area[area.weekday == 1]
        area3 = area[area.weekday == 2]
        area4 = area[area.weekday == 3]
        area5 = area[area.weekday == 4]
        result.append(len(area1)+len(area2)+len(area3)+len(area4)+len(area5))    
        i=i+1
    return result

In [69]:
resultWeekday = checkWeekday(allArea,allSanFran)

In [70]:
allArea['RecordWeek'] = resultWeekday

In [71]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AverageAcceleration_m/s^2,NumberTotalPointInterest,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",RecordWeekend,RecordWeek
0,9q8qy,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
1,9q9nb,1,0,28,96.950243,23.559269,1,0,0,8,...,0.217249,0,0,0,0,0,0,0,27,1
2,9q8qx,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
3,9q9p0,39,22,30498,149.903924,71.420760,39,22,7758,8233,...,0.379798,0,0,0,0,0,0,0,9200,21298
4,9q8qz,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
5,9q8rn,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
6,9q8zm,0,0,487,149.485681,19.056816,0,0,214,120,...,0.307872,14,1,5,1,0,1,6,244,243
7,9q8zq,0,0,4,101.578811,71.671195,0,0,1,3,...,0.350905,0,0,0,0,0,0,0,0,4
8,9q8yg,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
9,9q8zw,0,0,2,109.357496,72.297081,0,0,1,1,...,0.350307,0,0,0,0,0,0,0,0,2


In [73]:
allArea.to_csv('./sanFrancisco.csv')

In [ ]:
#prova a cambiare il geohash
#leggo nuovamente da file i 3 dataset

allArea = pd.read

,driver,lat,lon,occupied,timestamp
0,egteir,37.75135,-122.39526,0,1212567954
1,egteir,37.75143,-122.39529,0,1212567848
2,egteir,37.75154,-122.39531,0,1212567624
3,egteir,37.75154,-122.39531,0,1212567577
4,egteir,37.75159,-122.39530,0,1212567509
5,egteir,37.75202,-122.39467,0,1212567444
6,egteir,37.74992,-122.39339,0,1212567384
7,egteir,37.74884,-122.40372,0,1212567323
8,egteir,37.75952,-122.40633,0,1212567263
9,egteir,37.76916,-122.41008,0,1212567202
